* Data downloaded from : https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR.html
* Data dictionaries (and descs of other tables): https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR/dbinfo.html


```
PREDICATION table
Each record in this table identifies a unique predication. The data fields are as follows:

PREDICATION_ID: Auto-generated primary key for each unique predication
SENTENCE_ID: Foreign key to the SENTENCE table
PMID: The PubMed identifier of the citation to which the predication belongs
PREDICATE: The string representation of each predicate (for example TREATS, PROCESS_OF)
SUBJECT_CUI: The CUI of the subject of the predication
SUBJECT_NAME: The preferred name of the subject of the predication
SUBJECT_SEMTYPE: The semantic type of the subject of the predication
SUBJECT_NOVELTY: The novelty of the subject of the predication
OBJECT_CUI: The CUI of the object of the predication
OBJECT_NAME: The preferred name of the object of the predication
OBJECT_SEMTYPE: The semantic type of the object of the predication
OBJECT_NOVELTY: The novelty of the object of the predication
```
----------
```
GENERIC_CONCEPT table
This table contains the UMLS Metathesaurus concepts that are considered too generic based upon the 2006AA release. Concepts that are not stored in this table are considered novel. This table is used to populate the SUBJECT_NOVELTY and OBJECT_NOVELTY columns in the PREDICATION table defined below. Data fields in this table are as follows:

CONCEPT_ID: Auto generated primary key for each concept
CUI: The Concept Unique Identifier (CUI)
PREFERRED_NAME: The preferred name of the concept
1956C0699748Pathogenesis
```


* TODO : there are CUIs and concepts with "|" in them - e.g. C0034818|3643	Insulin Receptor|INSR	 , and many genes. How to handle/explode? 
I ignore them for now and drop them in subsequent steps. 

In [1]:
import pandas as pd

SAVE = True
FAST = False

# SAVE =False
# FAST = True

FILTER_MIN_PAIR_COOC_THRESHHOLD = 2 ## remove pairs from KG that have less than this many suppoting documents/PMIDs. 

In [2]:
%%time
### citations - contains PMID and dates/year for things. Useful for tempora l splitting

df_cite = pd.read_csv("semmedVER43_CITATIONS.csv.gz",header=None,
                      names = ["PMID","ISSN","DP","EDAT","YEAR"],usecols=["PMID","YEAR"],
                      nrows= 5e6 if FAST else None
                     ).set_index("PMID")
df_cite

CPU times: user 4.86 s, sys: 275 ms, total: 5.14 s
Wall time: 7.8 s


,YEAR
PMID,
1,1975
10,1975
100,1975
1000,1975
10000,1976
...,...
9999995,1991
9999996,1991
9999997,1991


In [ ]:
%%time
# 3/17 min
df = pd.read_csv("semmedVER43_PREDICATION.csv.gz", encoding = "ISO-8859-1", # try
                 nrows=2e5 if FAST else None,
                 skip_blank_lines=True,na_values='\\N',
# usecols = [
    # "PMID", "PREDICATE", "SUBJECT_CUI", "SUBJECT_NAME", "SUBJECT_SEMTYPE", 
    # "SUBJECT_NOVELTY", "OBJECT_CUI", "OBJECT_NAME", "OBJECT_SEMTYPE", "OBJECT_NOVELTY"
# ]
 ).dropna(axis=1,how="all") # diff # cols detected

df.columns = ["PREDICATION_ID",
"SENTENCE_ID",
"PMID",
"PREDICATE",
"SUBJECT_CUI",
"SUBJECT_NAME",
"SUBJECT_SEMTYPE",
"SUBJECT_NOVELTY",
"OBJECT_CUI",
"OBJECT_NAME",
"OBJECT_SEMTYPE",
"OBJECT_NOVELTY"]

df = df[["SUBJECT_CUI", "SUBJECT_NAME",  "PREDICATE",
      "PMID", ## drop or use agg counts for confidence?
         # "SUBJECT_SEMTYPE","OBJECT_SEMTYPE", ## semtype not used currently - skip it 
    "SUBJECT_NOVELTY", "OBJECT_CUI", "OBJECT_NAME",  "OBJECT_NOVELTY"]].drop_duplicates()
print(df.shape[0])
## ORIG: df = df.loc[df.groupby("PREDICATE").transform("size")>=100].copy() 
df = df.loc[df.groupby("PREDICATE")["SUBJECT_CUI"].transform("count")>=50].reset_index(drop=True).copy() 
print(df.shape[0],"After dropping rare predicates")
# ## add?
# df = df.loc[(df.groupby("OBJECT_CUI")["SUBJECT_CUI"].transform("size")>=2)\
# & (df.groupby("SUBJECT_CUI")["OBJECT_CUI"].transform("size")>=2)].copy()
# print(df.shape[0],"After dropping rare CUIs")
df

In [ ]:
df.dtypes

#### multi terms not fixed yet
* Also, may be enough to take first term per "|" list? Some are different terms maybe , some seem like synonyms, e.g. NPY/Neuropeptide Y 

In [ ]:
## look at cases with "|" in them, in subject... 
"""
data_updated = df.loc[df["SUBJECT_CUI"].str.contains("|",na=False,regex=False)]
display(data_updated)

# Split the SUBJECT_CUI and SUBJECT_NAME columns, handling NaN values appropriately
split_cui = data_updated['SUBJECT_CUI'].str.split('|').apply(lambda x: x if isinstance(x, list) else [])
split_name = data_updated['SUBJECT_NAME'].str.split('|').apply(lambda x: x if isinstance(x, list) else [])

# Explode the DataFrame, preserving rows even if they have empty lists from NaN entries
exploded_data_updated = data_updated.loc[data_updated.index.repeat(split_cui.apply(len))].copy()
exploded_data_updated['SUBJECT_CUI'] = pd.Series([item for sublist in split_cui for item in sublist])
exploded_data_updated['SUBJECT_NAME'] = pd.Series([item for sublist in split_name for item in sublist])

# Function to adjust CUIs, handling missing or incorrect values
def correct_adjust_cui(cui):
    if pd.isna(cui):
        return cui  # Keep NaN as NaN
    if not str(cui).startswith('C'):
        return 'C' + cui
    return cui

# Apply the corrected CUI function to ensure all CUIs are correctly formatted
exploded_data_updated['SUBJECT_CUI'] = exploded_data_updated['SUBJECT_CUI'].apply(correct_adjust_cui)

# Drop any duplicates and reset index
final_data_updated = exploded_data_updated.drop_duplicates().reset_index(drop=True)

# Show the updated DataFrame and check for NaN handling
final_data_updated
"""

In [ ]:
print(df_cite.shape[0])
df_cite = df[["PMID","SUBJECT_CUI","OBJECT_CUI","PREDICATE"]].drop_duplicates().merge(df_cite,on="PMID")
print(df_cite.shape[0])
print(df_cite.nunique())
### Get earliest year of a SVO occurring, or a pair occurring 

df_cite["first_year_pair"] = df_cite.groupby(["SUBJECT_CUI","OBJECT_CUI"])["YEAR"].transform("min")
df_cite["first_year_triple"] = df_cite.groupby(["SUBJECT_CUI","OBJECT_CUI","PREDICATE"])["YEAR"].transform("min")

if SAVE:
    df_cite.to_parquet("kg_first_cite_history_raw.parquet")
df_cite

In [ ]:
df_cite.groupby(["SUBJECT_CUI","OBJECT_CUI"])["YEAR"].transform("nunique")

### drop rareties: S-O level (pairs)

In [ ]:
df2 = df_cite[["SUBJECT_CUI","OBJECT_CUI","PMID"]].drop_duplicates().copy()
df2["pair_counts"] = df2.groupby(["SUBJECT_CUI","OBJECT_CUI"],observed=True)["PMID"].transform("nunique")

df2 = df2.drop(columns=["PMID"]).drop_duplicates()
print(df2.shape[0],"# pairwise count rows of subject/object")

df2 = df2.loc[df2["pair_counts"]>=FILTER_MIN_PAIR_COOC_THRESHHOLD]
print(df2.shape[0],"# pairwise count rows of subject/object, with >1 occurence")
# df2["pair_counts"] = df2.groupby(["SUBJECT_CUI","OBJECT_CUI"],observed=True)["PMID"].nunique()

s1 = df_cite.shape[0]
print(s1,"prev df_cite size")
df_cite = df_cite.merge(df2,on=["SUBJECT_CUI","OBJECT_CUI"],how="inner")
assert s1>= df_cite.shape[0] ,df_cite.shape[0]
print(df_cite.shape[0])

In [ ]:
print(df.shape[0])
df = df.merge(df_cite[["SUBJECT_CUI","OBJECT_CUI","PREDICATE","first_year_pair","first_year_triple","pair_counts"]].drop_duplicates(),
             on=["SUBJECT_CUI","OBJECT_CUI","PREDICATE"])
print(df.shape[0])

# ## add new - pairwise counts (ignores predicate). 
# ## Could maybe not count "NEG_" predicates for this purpose? 
# df["pair_counts"] = df.groupby(["SUBJECT_CUI","OBJECT_CUI"],observed=True)["SUBJECT_NOVELTY"].transform("size")
del df_cite, df2
df

In [ ]:
# df = df.merge(df2,on=["SUBJECT_CUI","OBJECT_CUI"]) ## add pair_counts - # unique papers a pair appeared in. 
# print(df.shape[0])
# del df_cite, df2

In [ ]:
%%time
print(df.shape[0])
df = df.drop_duplicates().drop(columns=["PMID"]) # 1 case max per work - Now will also need dropping years..

print(df.shape[0])

In [ ]:
df.info()

* Q: Note: more names than unique CUIS

In [ ]:
df.nunique()

In [ ]:
## (new) comment out here, we do it anyway below 
# for c in df.select_dtypes("O").columns:
#     df[c] = df[c].astype("category")
df["OBJECT_NOVELTY"] = df["OBJECT_NOVELTY"].astype(bool)
df["SUBJECT_NOVELTY"] = df["SUBJECT_NOVELTY"].astype(bool)
df.info()    

In [ ]:
print(df.drop_duplicates().shape[0])

##### Record # occurrences per triple, and keep max - at `SVO/triple` level

In [ ]:
%%time
# may be very slow
##
df["counts"] = df.groupby(["SUBJECT_CUI","PREDICATE","OBJECT_CUI"],observed=True)["SUBJECT_NOVELTY"].transform("size")


In [ ]:
%%time
df = df.sort_values(["counts","SUBJECT_CUI","PREDICATE","OBJECT_CUI"],ascending=False).drop_duplicates(subset=["PREDICATE",
                                                                                                               "SUBJECT_CUI","OBJECT_CUI",
                                                                                                               "SUBJECT_NAME","OBJECT_NAME",
                                                                                                               # "SUBJECT_NOVELTY","OBJECT_NOVELTY"
                                                                                                              ])
print(df.shape)

In [ ]:
df

In [ ]:
df["PREDICATE"].value_counts().tail(10)

In [ ]:
df["PREDICATE"].value_counts().head(25)

In [ ]:
df[df["SUBJECT_CUI"].str.contains(r"|",case=False,regex=False)].head(2)

In [ ]:
%%time
## new - 
##  ||| gene objecgts (that require exploding ) - handle them here in advance
splitter_mask = (df["SUBJECT_CUI"].str.contains(r"|",case=False,regex=False)) | (df["OBJECT_CUI"].str.contains(r"|",case=False,regex=False))
for col in df.select_dtypes(include=['category']).columns:
  df[col] = df[col].astype('str')
    
df.loc[splitter_mask,"SUBJECT_NAME"] = df.loc[splitter_mask]["SUBJECT_NAME"].str.split(r"|",regex=False,expand=True)[0]
df.loc[splitter_mask,"OBJECT_NAME"] = df.loc[splitter_mask]["OBJECT_NAME"].str.split(r"|",regex=False,expand=True)[0]

df.loc[splitter_mask,"SUBJECT_CUI"] = df.loc[splitter_mask]["SUBJECT_CUI"].str.split(r"|",regex=False,expand=True)[0]
df.loc[splitter_mask,"OBJECT_CUI"] = df.loc[splitter_mask]["OBJECT_CUI"].str.split(r"|",regex=False,expand=True)[0]
# duplicate of doing this above..
for c in df.select_dtypes("O").columns:
    df[c] = df[c].astype("category")
# df["OBJECT_NOVELTY"] = df["OBJECT_NOVELTY"].astype(bool)
# df["SUBJECT_NOVELTY"] = df["SUBJECT_NOVELTY"].astype(bool)
df.drop(columns=["OBJECT_NOVELTY","SUBJECT_NOVELTY","SUBJECT_SEMTYPE","OBJECT_SEMTYPE"],errors="ignore",inplace=True)

In [ ]:
# df[df["SUBJECT_CUI"].str.contains(r"|",case=False,regex=False)].head(3)

#### keep CUIs that appear at least 2 times in data/KG - per source cui?
#### Drop rare predicates
* Drops tiny (~<1%) amount of rows

In [ ]:
%%time
print(df.shape[0])
print(df["PREDICATE"].nunique())

df = df.loc[(df.groupby("OBJECT_CUI")["SUBJECT_CUI"].transform("size")>=2)\
& (df.groupby("SUBJECT_CUI")["OBJECT_CUI"].transform("size")>=2)].copy()
print(df.shape[0],"# rows after dropping singleton entities")

## pair counts filtering already done
## new - drop cases of a pair appearing only 1 time, regardless of predicate
df = df.loc[df["pair_counts"]>1]
print(df.shape[0],"# rows after dropping singleton pair counts")

## drop super rare/noise predicates
df = df.loc[df.groupby("PREDICATE").transform("size")>=50]#.copy() # 27,350,365  - very few cases of bad predicates

df = df.reset_index(drop=True)
for c in df.select_dtypes("category").columns:
# remove unobserved categories, in new filtered data
    df[c] = df[c].cat.remove_unused_categories()
print(df.shape[0])
print(df.nunique())

In [ ]:
df

In [ ]:
%%time
if SAVE:
    df.to_parquet("predications.parquet")

In [ ]:
df = pd.read_parquet("predications.parquet")
df

In [ ]:
# df.query("counts>2 & first_year_pair>1948")["first_year_pair"].hist()
df.query("counts>2 & first_year_pair>1948")["first_year_pair"].describe().round()